# Interactive KBO Notebook
In this notebook, we will make a bare-bones application to interactively explore the movement and light profile of KBOs and other moving objects. To fetch data, we will use the [Solar System Object Image Search](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/ssois/documentation.html) tool provided by the CADC.


## Table of Contents
* [1. Introduction](#1.-Introduction)
* [2. Setup](#2.-Setup)
    * [2.1 Using pip](#2.1-Using-pip)
    * [2.2 From source](#2.2-From-source)
* TODO


## 1. Introduction

The idea for this notebook is to give the outline of an interactive widget in a jupyter notebook. TODO: More info

## 2. Setup
This tutorial will go through some of the basic functionalities of the CADC astroquery package. The CADC module can be installed in two ways:

### 2.1 Using pip
The CADC module is only available with the pre-release of the astroquery module, and can be installed using the command:

```
    pip install --pre --upgrade astroquery
```

### 2.2 From source
Alternatively, you can clone and install from the source:
```
    # If you have a github account:
    git clone git@github.com:astropy/astroquery.git
    # If you do not:
    git clone https://github.com/astropy/astroquery.git
    cd astroquery
    python setup.py install
```
Note that these commands can also be done in a Jupyter notebook by either declaring the code cell a bash cell by pasting `%%bash` at the top of the cell, or preceding each line with a `!`. More information about astroquery can be found at the [astroquery github repository](https://github.com/astropy/astroquery). 



In [81]:
# ! conda activate tutorials
# !jupyter nbextension enable --py widgetsnbextension
# !jupyter nbextension enable --py --sys-prefix ipympl
# !jupyter nbextension enable --py --sys-prefix ipyaladin


In [82]:
import math
import numpy as np



from ipywidgets import Layout, Output, Box, widgets


from astropy.time import Time, TimeDelta
import astropy.units as u
from astroquery.cadc import Cadc


## SSOIS Query
In order to get images of a moving object, we will use the [Solar System Object Image Search](https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/ssois/documentation.html) tool provided by the CADC. We will search by object name between a start and end date. To choose an object to search, we will use the [Minor Planet Center search tool](https://minorplanetcenter.net/db_search/) and search the database where the minimum semi-major axis value is 50 AU. From the [results table](https://minorplanetcenter.net/db_search/show_by_properties?utf8=%E2%9C%93&semimajor_axis_min=50&semimajor_axis_max=&eccentricity_min=&eccentricity_max=&inclination_min=&inclination_max=&argument_of_perihelion_min=&argument_of_perihelion_max=&ascending_node_min=&ascending_node_max=&mean_anomaly_min=&mean_anomaly_max=&mean_daily_motion_min=&mean_daily_motion_max=&perihelion_distance_min=&perihelion_distance_max=&aphelion_distance_min=&aphelion_distance_max=&period_min=&period_max=&absolute_magnitude_min=&absolute_magnitude_max=&orbit_uncertainty_min=&orbit_uncertainty_max=), we choose [the KBO 2010 GB174](https://minorplanetcenter.net/db_search/show_object?object_id=2010+GB174) to query for.

In [93]:
import pandas as pd
from urllib.parse import urlencode

# Define search parameters
kbo_name = '2010 GB174'
start_date = '2010 01 01'
end_date = '2011 01 01'

# Build query url
params = {'lang': 'en', 'object': kbo_name, 'search': 'bynameCADC', 
          'epoch1': start_date, 'epoch2': end_date, 'eunits': 'arcseconds',
          'extres': 'yes', 'xyres': 'yes','format':'tsv'}
base_url = 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/cadcbin/ssos/ssosclf.pl'
url = '{}?{}'.format(base_url, urlencode(params))

# Access data and convert to panadas dataframe
data_table = pd.read_csv(url, sep='\t')

# Get results with positive extension values, a long exposure, and the g-band filter
data_table = data_table[(data_table['Ext'] > 0) & 
                        (data_table['Exptime'] > 100) & 
                        (data_table['Filter']=="G.MP9401")]

#table = Table.from_pandas(data_table)

#table
data_table

,Image,Ext,X,Y,MJD,Filter,Exptime,Object_RA,Object_Dec,Image_target,Telescope/Instrument,MetaData,Datalink
17,1183654p,18,2054.4,1047.4,55298.348651,G.MP9401,634,189.622392,15.045956,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
18,1183824p,18,2074.9,650.3,55299.294388,G.MP9401,634,189.611366,15.049740,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
19,1183831p,18,2017.1,1488.6,55299.351782,G.MP9401,634,189.610699,15.049966,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
20,1183838p,18,1917.8,324.0,55299.409538,G.MP9401,634,189.610028,15.050193,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN
21,1183845p,18,1935.4,1003.9,55299.466308,G.MP9401,634,189.609369,15.050416,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/da...,NaN


# IpyAladin Widget
To show the location of the results, we can use the interactive widget provided by Aladin. This widget has much more functionality that can be explored in the [jupyter notebooks on the ipyaladin github](https://github.com/cds-astro/ipyaladin/tree/master/examples).

In [94]:
from ipyaladin import Aladin

aladin = Aladin(layout=Layout(width='100%', height='400px'), fov=20)
aladin

Aladin(fov=20.0, layout=Layout(height='400px', width='100%'), options=['allow_full_zoomout', 'coo_frame', 'fov…

In [96]:
from astropy.table import Table

aladin.add_table(Table.from_pandas(data_table))

# Helper functions
The end goal is to make an application with a date slider that selects a subset of data then plots all the files in the subset, with interactivity to preform aperture photometry. To speed up the program, we can use cutouts to only show the data that we want to look at. In order to do all of this, we need helper functions to select a subset of images, calculate the cutout parameters, get the cutout urls, and fetch the cutout data.

In [86]:
import re
from astropy.wcs import WCS
from astropy.io import fits
from urllib.error import HTTPError

def get_selection(table, date_range):
    """Returns the subset of rows in the table within the date_range."""
    return table[(table['MJD'] >= date_range[0]) & 
                        (table['MJD'] < date_range[1]+1)]

def get_cutout_params(table, min_radius=0.005, ra_col_name='Object_RA', dec_col_name='Object_Dec'):
    """Given a table with R.A. and Dec. columns, returns the parameters of the
    circle needed to include all positions."""
    
    # Get the max and min R.A. and Dec. values
    ra_max, ra_min = table[ra_col_name].max(), table[ra_col_name].min()
    dec_max, dec_min = table[dec_col_name].max(), table[dec_col_name].min()

    # Get the R.A. and Dec. centre and radius that covers all points
    ra_centre = (ra_max + ra_min)/2 
    dec_centre = (dec_max + dec_min)/2
    ra_radius =  abs(ra_max - ra_min)/2 
    dec_radius =  abs(dec_max - dec_min)/2
    radius = max(round(max(ra_radius, dec_radius), 2), min_radius)
    
    return ra_centre, dec_centre, radius

def get_cutout_urls(url, ra, dec, radius=0.02):
    """Get the cutout urls around the given R.A., Dec., and radius."""
    
    uri = 'ad:' + \
        re.findall(
            "http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/(.+)\[", url)[0] + '.fits.fz'

    cutout_region_string = '{ra} {dec} {radius}'.format(
        ra=ra, dec=dec, radius=radius)
    params_dict = {'ID': uri, 'CIRCLE': cutout_region_string}

    base_url = 'https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/caom2ops/sync'

    url_params = urlencode(params_dict)
    data_url = '{}?{}'.format(base_url, url_params)
    return data_url


def get_data_list(table, ext=2):
    """Given the table with cutout urls, return the fits image and WCS information"""
    #TODO: Choose smarter extension rather than just ext=2 lol
    data_list = []
    if len(table) < 1:
        print("Range is too small")
        return None
    ra, dec, radius = get_cutout_params(table)
    table['cutout_url'] = [get_cutout_urls(url, ra, dec, radius=radius*2) for url in table['MetaData']]

    for url, ra, dec, date in zip(table['cutout_url'], table['Object_RA'], 
                                  table['Object_Dec'], table['MJD']):
        try:
            with fits.open(url, ignore_missing_end=True) as hdulist:
                data = hdulist[ext].data
                wcs = WCS(hdulist[ext].header)
                date_fmtd = Time(date, format='mjd', out_subfmt='date_hm').iso
                data_list.append({'image_data': data, 'wcs': wcs, 'ra': ra, 'dec': dec, 'date': date_fmtd})
        except HTTPError as ex:
            print(ex)
            continue
    return data_list

# Widgets
Now we make our widgets! We want one widget to allow the user to select a date range to display, and another widget to display the aperture data. Technically, we also want the plot itself to be a widget, but we will come to that later.

In [97]:
from traitlets import traitlets

# Date range slider widget
def make_date_range_slider(table):
    """ Function to generate the date range slider based on the date ranges in the results table"""
    start_date = math.floor(table['MJD'].min())
    end_date = math.ceil(table['MJD'].max())

    dates = Time(np.arange(start_date, end_date, 1), format='mjd', out_subfmt='date')
    # Match value to formated date output for nicer display
    options = [(' {} '.format(date.iso), date.mjd) for date in dates]
    
    # Range slider to select dates
    range_slider = widgets.SelectionRangeSlider(
        options=options,
        index=(0, 1),
        description='Date Range:',
        disabled=False,
        continuous_update=True,
        layout=Layout(width='90%'),
    )
    
    # The slider controls the selected rows of the table data
    selection = get_selection(table, range_slider.value)
    range_slider.add_traits(selection=traitlets.Any(selection))
        
    # Label to show number of files in selected date range
    range_label = widgets.Label(
        value="Number of Results: {}".format(len(selection))
    )
    
    # When slider changes, update selection table and results num label
    def update_selection(change):
        new_selection = get_selection(table, change['new'])
        range_slider.selection = new_selection
        range_label.value = "Number of Results: {}".format(len(new_selection))
        
    range_slider.observe(update_selection, names='value')
    
    return widgets.VBox([range_slider, range_label])

# Aperture results widget
def make_aperture_text_widget():
    """Returns widget to display aperture information"""
    aperture_widget = widgets.Label(
        placeholder='<p>Click and drag on a subplot to select an area to analyze</p>',
        description='Aperture Sum:',
        disabled=False,
        style={'description_width': 'initial'},
        layout=Layout(width='90%'),
    )
    return aperture_widget


## The KBO figure
This is the interactive plot of the KBO. Since the `matplotlib widget` backend is used, the figure canvas can be used as a widget, which we will use in a box along with the aperture widget. We want the ability to draw circles to show which area is going to be used in the aperture analysis, meaning we will have to use click event handlers.

In [88]:
%matplotlib widget
import matplotlib.pyplot as plt
import warnings
from matplotlib.patches import Circle

from photutils import CircularAperture, aperture_photometry
import matplotlib.gridspec as gridspec
from astropy.visualization import LinearStretch, ImageNormalize, ZScaleInterval
from IPython.display import display
from ipywidgets import interactive


# Supress fits processing warnings
warnings.simplefilter('ignore')
    

def kbo_figure(data_list, kbo_name):

    p_center = None
    global data_dict
    
    data_dict = {}
    
    def onclick(event):
        # Check if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != '' or event.inaxes is None): return

        global p_center

        #     if circle and circle.contains(event):
        #         circle.set_color('orange')
        p_center = (event.xdata, event.ydata)

    def onrelease(event):
        # Check if in pan/zoom mode or click outside of axes
        if (plt.get_current_fig_manager().toolbar.mode != '' or event.inaxes is None): return
    
        global p_center
        global kbo_fig
        global data_dict
        
        p_outside = (event.xdata, event.ydata)
        radius = math.sqrt((p_center[0] - p_outside[0])**2 +
                           (p_center[1] - p_outside[1])**2)
        
        if radius == 0.0: return
        
        # Remove old circles
        for ax in kbo_fig.axes:
            for artist in ax.artists:
                if isinstance(artist, plt.Circle):
                    artist.remove()

        # Draw new circles
        circle = plt.Circle(p_center, radius, color='black', fill=False)
        ax = event.inaxes
        ax.add_artist(circle)
        kbo_fig.canvas.draw()

        aperture = CircularAperture(p_center, r=radius)
        image_data = data_dict[ax.colNum]
        phot_table = aperture_photometry(image_data, aperture)
        aperture_widget.value = ' '.join(phot_table['aperture_sum'].pformat(html=True))

    def plot_data(kbo_fig, data_list, max_cols=3):
        n = len(data_list)
        global data_dict
        cid_press = kbo_fig.canvas.mpl_connect('button_press_event', onclick)
        cid_release = kbo_fig.canvas.mpl_connect('button_release_event', onrelease)
        wcs_trans = data_list[0]['wcs']
        gs = gridspec.GridSpec(math.ceil(n / max_cols), max_cols, 
                               figure=kbo_fig, wspace=0.1)

        for idx, data in enumerate(data_list):
            try:

                # Normalize and plot the data
                ax = plt.subplot(gs[idx//max_cols, idx%max_cols], 
                                 projection=wcs_trans, adjustable='box', aspect='equal')
                data_dict[idx] = data['image_data']

                image_data_norm = ImageNormalize(data['image_data'],
                                                 interval=ZScaleInterval(),
                                                 stretch=LinearStretch())
                ax.imshow(data['image_data'],
                          norm=image_data_norm,
                          transform=ax.get_transform(data['wcs']),
                          cmap='gray')
                
                ax.set_title(data['date'], fontsize=10)
                
                # Add yellow circle around KBO position
                c = Circle((data['ra'], data['dec']), 0.004, edgecolor='yellow', facecolor='none',
                      transform=ax.get_transform('world'))
                ax.add_patch(c)

                # Add grid and remove axis labels
                ax.coords.grid(color='white', ls='solid')
                ra, dec = ax.coords['ra'], ax.coords['dec']
                ra.set_ticklabel_visible(False)
                dec.set_ticklabel_visible(False)
                ra.set_axislabel('')
                dec.set_axislabel('')

            except ValueError as ex:
                print('Value Error: %s' % ex)
                continue

        axes = kbo_fig.get_axes()
        axes[0].get_shared_x_axes().join(*axes)
        axes[0].get_shared_y_axes().join(*axes)
        plt.subplots_adjust(left=0.05, right=0.95, top=0.9)
        plt.tight_layout(pad=0.5)

    kbo_fig = plt.figure(figsize=(9, 8))
    kbo_fig.suptitle('Images of KBO {}'.format(kbo_name), fontsize=14, fontweight='bold')
    plot_data(kbo_fig, data_list)
    return kbo_fig



In [89]:
range_slider = make_date_range_slider(table)
range_slider

In [90]:
selection = range_slider.children[0].selection
selection

Image,Ext,X,Y,MJD,Filter,Exptime,Object_RA,Object_Dec,Image_target,Telescope/Instrument,MetaData,Datalink
str8,int64,float64,float64,float64,str8,int64,float64,float64,str8,str12,str70,float64
1183654p,18,2054.4,1047.4,55298.3486508232,G.MP9401,634,189.62239200163899,15.0459562249869,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/1183654p[18],--
1183824p,18,2074.9,650.3,55299.294387719005,G.MP9401,634,189.611366469382,15.0497404406177,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/1183824p[18],--
1183831p,18,2017.1,1488.6,55299.351782286794,G.MP9401,634,189.61069949667598,15.049966033155,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/1183831p[18],--
1183838p,18,1917.8,324.0,55299.4095384231,G.MP9401,634,189.61002832224202,15.0501930468574,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/1183838p[18],--
1183845p,18,1935.4,1003.9,55299.4663084248,G.MP9401,634,189.609368607514,15.050416184503,NGVS+2+3,CFHT/MegaCam,http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/data/pub/CFHT/1183845p[18],--


In [91]:
plt.ioff()
data_list = get_data_list(range_slider.children[0].selection)
kbo_fig = kbo_figure(data_list, kbo_name)
plt.ion()


In [92]:
# Display Block
# Organize the widgets presentably
b_layout = widgets.Layout(align_items='center', align_content='center', border='none', justify_content = 'center', 
                          width = '100%')

aperture_widget = make_aperture_text_widget()
big_box = widgets.VBox([kbo_fig.canvas, aperture_widget], layout = b_layout)

display(big_box)


In [75]:
# TODO: Get date range slider to replot current fig (not make new figs!)
# TODO: Make aperture data more visually nice
# TODO: Fix WCS
# TODO: Fix aperture caluclation (rn not getting actually)